create tree with genre as a head and child as question and answer
create code to check the genre of specific question
create code to check if that genre is avaliable in tree if find than answer
if genre or question is not avaliabel in tree then simply ask the user for answer of particular question and save that question and answer in ai


# create through simple graphs

In [ ]:
class TreeNode:
    def __init__(self, question, answer, genre):
        self.question = question
        self.answer = answer
        self.genre = genre
        self.children = []

    def add_child(self, child):
        self.children.append(child)

In [ ]:


def search(root, query, genre):
    node = root
    while node:
        if node.genre == genre:
            for child in node.children:
                if child.question == query:
                    return child.answer
            break
        node = node.children[0]
    return "I don't know the answer to that question."


def create_tree():
    root = TreeNode("root", "root", "root")

    child1 = TreeNode("who is the first president of pakistan", "quaid-e-azam", "root")
    root.add_child(child1)

    return root


def determine_genre(query):
    if "first president" in query:
        return "root"
    return "unknown"


#  ------------------------------------
def main():
    tree = create_tree()

    while True:
        query = input("Ask a question: ")
        genre = determine_genre(query)
        answer = search(tree, query, genre)
        print(answer)

if __name__ == "__main__":
    main()

## create code for automatic detect genre of question

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def determine_genre(query):
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_query = [w for w in word_tokenize(query) if not w in stop_words]
    filtered_query = ' '.join(filtered_query)

    # Convert text to TF-IDF features
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([filtered_query])

    # Create a TF-IDF similarity matrix
    genre_tfidf_matrix = vectorizer.fit_transform(['first president'])
    genre_similarity = cosine_similarity(tfidf_matrix, genre_tfidf_matrix)

    if genre_similarity[0][0] > 0.7:
        return "root"
    return "unknown"

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

def determine_genre(query, genre_reference):
    # Remove stopwords
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    filtered_query = [w for w in word_tokenize(query) if not w in stop_words]
    filtered_query = ' '.join(filtered_query)

    # Convert text to TF-IDF features
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([filtered_query, genre_reference])

    # Calculate cosine similarity between the two vectors
    genre_similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    if genre_similarity[0][0] > 0.7:
        return "root"
    return "unknown"

# Example usage:
query = "Who was the first president?"
genre_reference = "first president"
result = determine_genre(query, genre_reference)
print(result)


In [ ]:
from transformers import pipeline

def determine_genre(sentence):
    # Load pre-trained text classification model
    classifier = pipeline('sentiment-analysis')

    # Use the model to predict the sentiment (genre in this case)
    result = classifier(sentence)

    # Extract the predicted label
    predicted_genre = result[0]['label']
    
    return predicted_genre

# Example usage:
user_input = "Who was the first president?"
result = determine_genre(user_input)
print(result)


In [ ]:
genre_key_phrases = {
    "root": ["first president", "presidential candidate", "leader"],
    "other_genre": ["example phrase", "another example"]
}
determine_genre(genre_key_phrases)

## 2 attempt

In [ ]:
data = [
    ("Who was the first president of pakistan?", "political"),
    ("What is the capital of France?", "geographical"),
    ("Which programming language is widely used for web development?", "technical"),
    # Add more examples...
]

In [ ]:
from simpletransformers.classification import ClassificationModel
model_args= {
            "num_train_epochs": 15,
            "learning_rate": 1e-5,
            "max_seq_length": 512,
            "silent": True
            }
model = ClassificationModel(
    "xlmroberta", "classla/xlm-roberta-base-multilingual-text-genre-classifier", use_cuda=True,
    args=model_args
    
)
predictions, logit_output = model.predict(["How to create a good text classification model? First step is to prepare good data. Make sure not to skip the exploratory data analysis. Pre-process the text if necessary for the task. The next step is to perform hyperparameter search to find the optimum hyperparameters. After fine-tuning the model, you should look into the predictions and analyze the model's performance. You might want to perform the post-processing of data as well and keep only reliable predictions.", 
                                        "On our site, you can find a great genre identification model which you can use for thousands of different tasks. With our model, you can fastly and reliably obtain high-quality genre predictions and explore which genres exist in your corpora. Available for free!"]
                                        )
predictions
# Output: array([3, 8])

[model.config.id2label[i] for i in predictions]
# Output: ['Instruction', 'Promotion']


###### create algo for other things

In [ ]:
import pandas as pd
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = {}

    def add_child(self, key, child_node):
        self.children[key] = child_node

class GenreTree:
    def __init__(self):
        self.root = TreeNode("root")

    def add_genre(self, genre):
        genre_node = TreeNode(genre)
        self.root.add_child(genre, genre_node)
        return genre_node

    def add_question_answer(self,genre , question, answer):
        if genre in  self.root.children:
            genre_node = self.root.children[genre]
        else:
            genre_node=self.add_genre(genre=genre)
            
        question_node = TreeNode(question)
        answer_node = TreeNode(answer)
        genre_node.add_child(question, (question_node, answer_node))
        
        data = {'Genre': [genre], 'Question': [question], 'Answer': [answer]}
        df = pd.DataFrame(data)
        try:
            existing_data = pd.read_excel('qa_data.xlsx')
            updated_data = pd.concat([existing_data, df], ignore_index=True)
            updated_data.to_excel('qa_data.xlsx', index=False)
        except FileNotFoundError:
            df.to_excel('qa_data.xlsx', index=False)
        

    def get_answer(self, genre, user_question):
        if genre in self.root.children:
            genre_node = self.root.children[genre]
            if user_question in genre_node.children:
                question_node, answer_node = genre_node.children[user_question]
                return answer_node.value
            else:
                answer=input("sorry i don't have this question in my database do you know the answer to this? \n kindly let me know if you did")
                if answer is not None:
                    self.add_question_answer(genre , user_question, answer)
                    print("Thanks for you feed back next i will response to this")
                else:
                    print("Thanks for your feed back")
                    
        else:
            return "I'm sorry, I don't have information on that genre."
        
        
    # def asking_for_adding_questions(self ,genre, user_question,answer):
    #     if genre in self.root.children:
    #         genre_node = self.root.children[genre]
    #         if user_question in genre_node.children:
    #             question_node, answer_node = genre_node.children[user_question]
    #             return answer_node.value
    #         else:
    #             answer=input("sorry i don't have this question in my dictionary do you know the answer to this? \n kindly let me know if you did")
    #             if answer is not None:
    #                 self.add_question_answer(self,genre , user_question, answer)
    #                 print("Thanks for you feed back next i will response to this")
    #     else:
    #         return "I'm sorry, I don't have information on that genre."
        

# Example usage:
genre_tree = GenreTree()

# Add genres
# history_genre = genre_tree.add_genre("history")
# science_genre = genre_tree.add_genre("science")

# Add questions and answers
genre_tree.add_question_answer('history', "Who was the first president?", "George Washington")
genre_tree.add_question_answer('science', "What is the capital of France?", "Paris")

# Get answers based on user input
user_genre = "history"

user_question = "Who was the first president?"
answer = genre_tree.get_answer(user_genre, user_question)
print(answer)

user_genre = "science"
user_question = "What is the capital of pakistan?"
answer = genre_tree.get_answer(user_genre, user_question)
print(answer)


In [ ]:
pd.read_csv("qa_data.csv")
data = {'Genre': ['genre'], 'Question': ['question'], 'Answer': ['answer']}
df = pd.DataFrame(data)

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = {}

    def add_child(self, key, child_node):
        self.children[key] = child_node

class GenreTree:
    def __init__(self, data_file='qa_data.csv'):
        self.root = TreeNode("root")
        self.data_file = data_file
        self.load_data()

    def load_data(self):
        try:
            self.qa_data = pd.read_csv(self.data_file,encoding='unicode_escape')
            for index, row in self.qa_data.iterrows():
                genre = row['Genre']
                question = row['Question']
                answer = row['Answer']
                self.add_question_answer(genre, question, answer)
        except FileNotFoundError:
            print(f"File '{self.data_file}' not found. No data loaded.1")

    def add_genre(self, genre):
        genre_node = TreeNode(genre)
        self.root.add_child(genre, genre_node)
        return genre_node

    def add_question_answer(self, genre, question, answer):
        if genre in self.root.children:
            genre_node = self.root.children[genre]
        else:
            genre_node = self.add_genre(genre=genre)
            question_node = TreeNode(question)
            answer_node = TreeNode(answer)
            genre_node.add_child(question, (question_node, answer_node))
            data = {'Genre': [genre], 'Question': [question], 'Answer': [answer]}
            df = pd.DataFrame(data)
            try:
                
                updated_data = pd.concat([self.qa_data, df], ignore_index=True)
                updated_data.to_csv('qa_data.csv', index=False)
            except FileNotFoundError:
                df.to_csv('qa_data.csv', index=False)

    def get_answer(self, genre, user_question):
        if genre in self.root.children:
            genre_node = self.root.children[genre]
            if user_question in genre_node.children:
                question_node, answer_node = genre_node.children[user_question]
                return answer_node.value
            else:
                answer = input("Sorry, I don't have this question in my database. Do you know the answer to this?\nKindly let me know if you do: ")
                if answer is not None:
                    self.add_question_answer(genre, user_question, answer)
                    print("Thanks for your feedback. I will respond to this next time.")
                else:
                    print("Thanks for your feedback.")
        else:
            return "I'm sorry, I don't have information on that genre."

    # def visualize_graph(self):
    #     G = nx.DiGraph()
    #     self._add_nodes_to_graph(self.root, G)
    #     self._add_edges_to_graph(self.root, G)
    #     pos = nx.spring_layout(G)
    #     nx.draw(G, pos, with_labels=True, font_weight='bold', arrowsize=20)
    #     plt.show()

    def _add_nodes_to_graph(self, node, graph):
        graph.add_node(node.value)
        for child_key, child_node in node.children.items():
            graph.add_node(child_key)
            self._add_nodes_to_graph(child_node[0], graph)  # Add question node

    def _add_edges_to_graph(self, node, graph):
        for child_key, child_node in node.children.items():
            graph.add_edge(node.value, child_key)
            self._add_edges_to_graph(child_node[0], graph)  # Add question node

# Example usage:
genre_tree = GenreTree()
genre_tree.add_question_answer('history', "Who was the first president?", "George Washington")
genre_tree.add_question_answer('science', "What is the capital of France?", "Paris")
# Get answers based on user input
user_genre = "history"
user_question = "Who was the first president?"
answer = genre_tree.get_answer(user_genre, user_question)
print(answer)

user_genre = "science"
user_question = "What is the capital of pakitan?"
answer = genre_tree.get_answer(user_genre, user_question)
print(answer)

# Visualize the graph
# genre_tree.visualize_graph()


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = {}

    def add_child(self, key, child_node):
        self.children[key] = child_node

class GenreTree:
    def __init__(self, data_file='qa_data.csv'):
        self.root = TreeNode("root")
        self.data_file = data_file
        self.load_data()

    def load_data(self):
        try:
            self.qa_data = pd.read_csv(self.data_file, encoding='unicode_escape')
            for index, row in self.qa_data.iterrows():
                genre = row['Genre']
                question = row['Question']
                answer = row['Answer']
                self.add_question_answer(genre, question, answer)
        except FileNotFoundError:
            print(f"File '{self.data_file}' not found. No data loaded.")

    def add_genre(self, genre):
        genre_node = TreeNode(genre)
        self.root.add_child(genre, genre_node)
        return genre_node

    def question_exists(self, genre, question):
        if genre in self.root.children and question in self.root.children[genre].children:
            return True
        return False

    def add_question_answer(self, genre, question, answer):
        if self.question_exists(genre, question):
            print(f"The question '{question}' already exists in the database.")
            return

        if genre in self.root.children:
            genre_node = self.root.children[genre]
        else:
            genre_node = self.add_genre(genre=genre)

        question_node = TreeNode(question)
        answer_node = TreeNode(answer)
        genre_node.add_child(question, (question_node, answer_node))

        data = {'Genre': [genre], 'Question': [question], 'Answer': [answer]}
        df = pd.DataFrame(data)
        try:
            updated_data = pd.concat([self.qa_data, df], ignore_index=True)
            updated_data.to_csv('qa_data.csv', index=False)
        except FileNotFoundError:
            df.to_csv('qa_data.csv', index=False)

    def get_answer(self, genre, user_question):
        if genre in self.root.children:
            genre_node = self.root.children[genre]
            if user_question in genre_node.children:
                question_node, answer_node = genre_node.children[user_question]
                return answer_node.value
            else:
                answer = input("Sorry, I don't have this question in my database. Do you know the answer to this?\nKindly let me know if you do: ")
                if answer is not None:
                    self.add_question_answer(genre, user_question, answer)
                    print("Thanks for your feedback. I will respond to this next time.")
                else:
                    print("Thanks for your feedback.")
        else:
            return "I'm sorry, I don't have information on that genre."

# Example usage:
genre_tree = GenreTree()
genre_tree.add_question_answer('history', "Who was the first president?", "George Washington")
genre_tree.add_question_answer('science', "What is the capital of France?", "Paris")
# Try adding the same question again (it should print a message that the question already exists)
genre_tree.add_question_answer('history', "Who was the first president?", "George Washington")

# Get answers based on user input
user_genre = "history"
user_question = "Who was the first president?"
answer = genre_tree.get_answer(user_genre, user_question)
print(answer)

user_genre = "science"
user_question = "What is the capital of Pakistan?"
answer = genre_tree.get_answer(user_genre, user_question)
print(answer)


In [ ]:
import random
rules = [
    ("what is the cost of {item}?", ["The cost of {item} is ${price}.", "For {item}, the cost is ${price}."]),
    ("can i afford to buy {item}?", ["Yes, you can afford to buy {item}.", "It is possible for you to buy {item}."]),
    ("i can't afford to buy {item} right now.", ["Don't worry, there are many other options available.", "Feel free to browse and compare prices before making a decision."]),
    ("i'm not sure if i can afford {item}.", ["You can use our calculator to help you determine your budget.", "Don't worry, we're here to help you find the best option."]),
    ("help me find something affordable.", ["You can filter your search by price to find more affordable options.", "Don't hesitate to ask for assistance if you need it."]),
]
def handle_response(user_input, item, price):
    for condition, responses in rules:
        if condition.format(item=item, price=price) in user_input:
            return random.choice(responses)

    # Default response if no matching rule is found
    return "I'm not sure how to help you with that."
# Replace these with your own item and price
item = "a laptop"
price = 1200

# Replace this with a user's input
user_input = "what is the cost of a laptop?"

response = handle_response(user_input, item, price)
print(response)